# Assignment 2: Question Paragraph Matching

Build a text vectorizer that finds the best matching paragraph for the provided set of questions and qualitatively evaluates the results

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [3]:
# Remove column width limit
pd.set_option('display.max_colwidth', None)

# Optional: show all columns (if you have many)
pd.set_option('display.max_columns', None)

# Optional: show all rows
pd.set_option('display.max_rows', None)

## Load paragraphs

In [4]:
df = pd.read_csv("data/paragraphs.csv")
df.head()

,paragraph
0,"2 About this report This document includes an updated Part B Methodology from the June 2019 report and an updated Part F Full list of technical screening criteria. The other original sections from the June 2019 report can be found as labelled in the June 2019 report. PART A Explanation of the Taxonomy approach. This section sets out the role and importance of sustainable finance in Europe from a policy and investment perspective, the rationale for the development of an EU Taxonomy, the daft regulation and the mandate of the TEG. PART B Methodology. This explains the methodologies for developing technical screening criteria for climate change mitigation objectives, adaptation objectives and do no significant harm to other environmental objectives in the legislative proposal. This has been updated since 2019. PART C Taxonomy user and use case analysis. This section provides practical guidance to potential users of the Taxonomy, including case studies. PART D Economic impacts of the Taxonomy. This section provides the TEGs analysis of the likely economic impacts of establishing an EU Taxonomy. PART E Next steps for the Taxonomy. This section elaborates on unresolved issues and potential ways forward for the Taxonomy and the technical work of the Platform on Sustainable Finance. PART F Full list of technical screening criteria. This annex sets out the sector and economic activityspecific technical screening criteria and rationale for the TEGs analysis. These have been updated since 2019."
1,"Disclaimer This report represents the overall view of the members of the Technical Expert Group, and although it represents such a consensus, it may not necessarily, on all details, represent the individual views of member institutions or experts. The views reflected in this Report are the views of the experts only. This report does not reflect the views of the European Commission or its services."
2,"3 Contents Methodology statements ....................................................................10 1. Substantial contribution to Climate change mitigation .......................................................... 10 1.1 Work process conceptual approach ............................................................................ 10 1.2 Defining substantial contribution to climate change mitigation ....................................... 14 1.3 Eligibility of finance for activities contributing substantially to mitigation ........................ 16 1.4 Further development ....................................................................................................... 16 2. Substantial contribution to Climate change adaptation ......................................................... 18 2.1 Work process conceptual approach ............................................................................ 18 2.2 Defining substantial contribution to climate change adaptation ..................................... 20 2.3 Screening criteria for activities making a substantial contribution to adaptation ............ 25 2.4 Eligibility of finance for activities contributing substantially to adaptation ....................... 27 2.5 Classification of climaterelated hazards ........................................................................ 28 3. Do no significant harm (DNSH) ............................................................................................. 29 3.1 DNSH to environmental objectives 36 ........................................................................... 29 3.2 DNSH to climate change adaptation ............................................................................... 29 3.3 DNSH to environmental objectives 36 ........................................................................... 31 Eligible NACE Codes Points of Note .................................................36 3.4 Use of NACE ...........................................................................................................

## Tf-Idf

todo: filter out stopwords 

In [17]:
vectorizer = TfidfVectorizer(min_df=1)

In [24]:
vector_corpus = vectorizer.fit_transform(df["paragraph"])


In [ ]:
vectors = vector_corpus.todense(),round(2)

In [25]:
print (df.shape)

(1234, 1)


In [21]:
print (vectors[0].shape)

(1234, 7031)


In [22]:
questions = [
    ["What fuel is used for manufacturing of chlorine?"],
    ["What metric is used for evaluating emission?"],
    ["How can carbon emission of the processes of cement clinker be reduced?"],
    ["How is the Weighted Cogeneration Threshold calculated?"],
    ["What is carbon capture and sequestration?"],
    ["What stages does CCS consist of?"],
    ["What should be the average energy consumption of a water supply system?"],
    ["What are examples of sludge treatments?"],
    ["How is the process of anaerobic digestion?"],
    ["How is reforestation defined?"],
    ["What is the threshold of emssion for inland passenger water transport?"], 
    ["What are the requirements of reporting for electricity generation from natural gas where there might be fugative emissions?"]
]

 use the linear kernal to calculate the distance between the question representation and the corpus vectors

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

qcontexts = []
for question in questions:
    # create a query_vector from the question
    q_v = vectorizer.transform(question)

    # use linear_kernel to find closest paragraphs
    lk_rank = linear_kernel(q_v, vector_corpus).flatten()

    # latest is closest
    paragraph_index = lk_rank.argsort()[-1]
    closest_paragraph = df["paragraph"][paragraph_index]
    qcontexts.append((question, closest_paragraph))

In [29]:
qcontexts

[(['What fuel is used for manufacturing of chlorine?'],
  'Do no significant harm assessment Manufacture of chlorine  The main potential significant harm to other environmental objectives from the manufacture of chlorine is associated with  polluting emissions to air (e.g. chlorine)  process water effluents which can contain oxidizing agents (e.g. chlorine)  the use of water in water stressed areas and  the generation of wastes Due to the intrinsic hazard properties of chlorine it is recommended to further assess when Chlorine could be considered part of the solution to achieving  zero pollution (toxic free environment) and therefore should not excluded from the taxonomy due to DNSH implications. (2) Adaptation   Refer to the screening criteria for DNSH to climate change adaptation.  (3) Water   Identify and manage risks related to water quality andor water consumption at the appropriate level. Ensure that water useconservation management plans, developed in consultation with relevant 

## compare results against gensim

In [33]:
import gensim

def read_corpus(text, tokens_only=False):
    for i, line in enumerate(text):
        tokens = gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

corpus = list(read_corpus(df["paragraph"].values))
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)
qcontext_doc2vec = []
for question in questions:
    q1 = list(read_corpus(question, tokens_only=True))
    inferred_vector = model.infer_vector(q1[0])
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    qcontext_doc2vec.append((question, df["paragraph"][sims[0][0]]))

In [34]:
for ic,(question, context)  in enumerate(qcontexts):
    print(question[0])
    print(f"tfidf: {context}\n\ndoc2vec: {qcontext_doc2vec[ic][1]}")
    print("-"*20)

What fuel is used for manufacturing of chlorine?
tfidf: Do no significant harm assessment Manufacture of chlorine  The main potential significant harm to other environmental objectives from the manufacture of chlorine is associated with  polluting emissions to air (e.g. chlorine)  process water effluents which can contain oxidizing agents (e.g. chlorine)  the use of water in water stressed areas and  the generation of wastes Due to the intrinsic hazard properties of chlorine it is recommended to further assess when Chlorine could be considered part of the solution to achieving  zero pollution (toxic free environment) and therefore should not excluded from the taxonomy due to DNSH implications. (2) Adaptation   Refer to the screening criteria for DNSH to climate change adaptation.  (3) Water   Identify and manage risks related to water quality andor water consumption at the appropriate level. Ensure that water useconservation management plans, developed in consultation with relevant sta